In [3]:
import torch
import MySQLdb
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.utils.data.sampler as smp
from tqdm import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import IPython.display
from torch.autograd import Variable
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models

import dataset
import dataLoader
import configure as cf

In [4]:
#define transform function, define trainset and valset
#AlexNet requires the input size of 224*224*3
imgTransform = transforms.Compose([transforms.Scale((224)),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor()                       
                            ])
trainLoader, valLoader = dataLoader.get_train_valid_loader(cf.photo_url,50,32,'food',imgTransform,0.1,-1)

food



  1%|          | 729/121267 [00:00<00:16, 7288.83it/s]

After search photo, find result: 121267
Start search stars for each photo.



 21%|██        | 25724/121267 [00:03<00:13, 7318.52it/s]

 84%|████████▎ | 101313/121267 [00:13<00:02, 7250.60it/s]


100%|██████████| 121267/121267 [00:16<00:00, 7272.67it/s]

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
def plot_loss(train_loss,val_loss):
    plt.plot(train_loss,'r',label = 'train loss')
    plt.plot(val_loss,'b',label = 'validation loss')
    plt.xlabel('epoch')
    plt.ylabel('loss scores')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

def plot_accuracy(train, val):
    plt.plot(train,'r',label = 'train accuracy')
    plt.plot(val,'b',label = 'validation accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy scores')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [6]:
#define AlexNet CNN class
class AlexNet(nn.Module):

    def __init__(self, num_classes=1):
        #change the num_classes 1000 to 1
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
    


#define train model
def train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 10, use_gpu = True):
    
    train_accuracy = []
    train_loss = []
    val_accuracy = []
    val_loss = []
    
    if use_gpu:
        network = network.cuda()
        criterion = criterion.cuda()
        
    # Training loop.
    for epoch in range(0, n_epochs):
        correct = 0.0
        cum_loss = 0.0
        counter = 0

        # Make a pass over the training data.
        t = tqdm(trainLoader, desc = 'Training epoch %d' % epoch)
        network.train()  # This is important to call before training!
        for (i, (inputs, stars)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            stars = Variable(stars.type(torch.FloatTensor))
            if inputs.size(0)<50 or stars.size(0)<50: break
            
            if use_gpu:
                inputs = inputs.cuda()
                stars = stars.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, stars)

            # Backward pass:
            optimizer.zero_grad()
            # Loss is a variable, and calling backward on a Variable will
            # compute all the gradients that lead to that Variable taking on its
            # current value.
            loss.backward() 

            # Weight and bias updates.
            optimizer.step()

            # logging information.
            #set a rule: if prediction values is between real_value-0.5 and real_value+0.5, correct+1
            cum_loss += loss.data[0]
            pre_star = outputs.data
            larger = (pre_star.view(50) >= (stars.data-0.5)).type(torch.IntTensor)
            littler = (pre_star.view(50) <= (stars.data+0.5)).type(torch.IntTensor)
            correct += (larger+littler).eq(2).sum() 
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
        train_accuracy.append(100 * correct / counter)
        train_loss.append(cum_loss / (1 + i))

        # Make a pass over the validation data.
        correct = 0.0
        cum_loss = 0.0
        counter = 0
        t = tqdm(valLoader, desc = 'Validation epoch %d' % epoch)
        network.eval()  # This is important to call before evaluating!
        for (i, (inputs, stars)) in enumerate(t):

            # Wrap inputs, and targets into torch.autograd.Variable types.
            inputs = Variable(inputs)
            stars = Variable(stars.type(torch.FloatTensor))
            if inputs.size(0)<50 or stars.size(0)<50: break
            
            if use_gpu:
                inputs = inputs.cuda()
                stars = stars.cuda()

            # Forward pass:
            outputs = network(inputs)
            loss = criterion(outputs, stars)

            # logging information.
            cum_loss += loss.data[0]
            pre_star = outputs.data
            larger = (pre_star.view(50) >= (stars.data-0.5)).type(torch.IntTensor)
            littler = (pre_star.view(50) <= (stars.data+0.5)).type(torch.IntTensor)
            correct += (larger+littler).eq(2).sum() 
            counter += inputs.size(0)
            t.set_postfix(loss = cum_loss / (1 + i), accuracy = 100 * correct / counter)
        val_accuracy.append(100 * correct / counter)
        val_loss.append(cum_loss / (1 + i))
    return [train_accuracy,val_accuracy,train_loss,val_loss]
            
#define learningRate
learningRate = 1e-2 

# Definition of our network.
# network = models.alexnet(pretrained = True, num_classes=1)
network = AlexNet()

#Definition of our loss.
#The MSELoss function (input? output?)
criterion = nn.MSELoss()

# Definition of optimization strategy.
optimizer = optim.SGD(network.parameters(), lr = learningRate)

result = []
# Train the previously defined model.
result = train_model(network, criterion, optimizer, trainLoader, valLoader, n_epochs = 20, use_gpu = True)

Training epoch 0:   0%|          | 2/2183 [00:04<1:49:33,  3.01s/it, accuracy=0, loss=13.6]

torch.Size([50, 1])
torch.Size([50])
torch.Size([50, 1])
torch.Size([50])


Training epoch 0:   0%|          | 3/2183 [00:04<1:18:51,  2.17s/it, accuracy=0, loss=13.2]

torch.Size([50, 1])
torch.Size([50])
torch.Size([50, 1])
torch.Size([50])


Training epoch 0:   0%|          | 6/2183 [00:05<31:05,  1.17it/s, accuracy=0, loss=11.8]  

...

torch.Size([50, 1])
torch.Size([50])
torch.Size([50, 1])
torch.Size([50])


Training epoch 0:   2%|▏         | 39/2183 [00:21<16:27,  2.17it/s, accuracy=0.718, loss=17.3]Process Process-4:
KeyboardInterrupt
Process Process-1:
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Process-2:
    self.run()
    self.run()
    self.run()
Traceback (most recent call last):
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/ho

torch.Size([50, 1])
torch.Size([50])


    self.load()
Exception     self.load()
    image = Image.open(img_address).convert('RGB')
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/site-packages/PIL/ImageFile.py", line 234, in load
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/site-packages/PIL/ImageFile.py", line 212, in load
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/site-packages/PIL/ImageFile.py", line 234, in load
  File "/home/aimin52qinhao/anaconda2/lib/python2.7/site-packages/PIL/Image.py", line 2419, in open
    n, err_code = decoder.decode(b)
    prefix = fp.read(16)
    n, err_code = decoder.decode(b)
    s = read(self.decodermaxblock)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
_mysql_exceptions.ProgrammingError: (2014, "Commands out of sync; you can't run this command now") in <bound method Cursor.__del__ of <MySQLdb.cursors.Cursor object at 0x7f16d5efd810>> ignored


KeyboardInterrupt: 